<a href="https://colab.research.google.com/github/Jonathan-C-Barrett/GEOG5990M/blob/main/GEOG5003M_Final_Project_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5003M Final Assignment (Template)

Student ID number:

GitHub repo link:

Word count limit= 1,500 words max (markdown cells only, excluding readme)

In [1]:
# read in required packages

import geopandas as gpd
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

import matplotlib.pyplot as plt

!pip install mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 10.8 MB/s eta 0:00:00


In [2]:
#Read in Health Services Data
ahah = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/ahah_v4.csv')
ahah.head()

,LSOA21CD,ah4blue,ah4dent,ah4gp,ah4hosp,ah4phar,ah4leis,ah4pubs,ah4ffood,ah4gpas,...,ah4g_rnk,ah4e_rnk,ah4r_rnk,ah4h_pct,ah4g_pct,ah4e_pct,ah4r_pct,ah4ahah,ah4ahah_rnk,ah4ahah_pct
0,E01000001,1.8048,0.6328,1.0316,0.9581,0.7487,0.3596,0.4210,0.3920,0.1056,...,40830,42589,41495,2,96,100,98,59.7543,42592,100
1,E01000002,2.1041,0.6230,1.3232,1.1370,0.6849,0.5571,0.4852,0.3946,0.0778,...,42201,42590,41673,4,99,100,98,65.4344,42635,100
2,E01000003,2.2753,0.5709,0.8562,0.7862,0.5919,0.4608,0.5679,0.5919,0.1339,...,40631,42590,40857,1,96,100,96,57.4313,42559,100
3,E01000005,1.1553,0.8337,1.1078,0.8697,0.6657,0.3717,0.2673,0.2934,0.0774,...,40765,42616,42419,2,96,100,100,64.3384,42629,100
4,E01000006,1.9055,1.9361,0.9587,1.5191,1.3641,1.5219,1.5017,1.1499,0.3310,...,23517,41369,34909,22,56,98,82,33.5326,38389,91


In [3]:
#Select only columns relating to Health Services
health = ahah[['LSOA21CD', 'ah4dent', 'ah4gp', 'ah4hosp', 'ah4phar', 'ah4leis', 'ah4h']]
health.head()

,LSOA21CD,ah4dent,ah4gp,ah4hosp,ah4phar,ah4leis,ah4h
0,E01000001,0.6328,1.0316,0.9581,0.7487,0.3596,-1.7834
1,E01000002,0.6230,1.3232,1.1370,0.6849,0.5571,-1.5523
2,E01000003,0.5709,0.8562,0.7862,0.5919,0.4608,-1.9813
3,E01000005,0.8337,1.1078,0.8697,0.6657,0.3717,-1.7260
4,E01000006,1.9361,0.9587,1.5191,1.3641,1.5219,-0.6686


In [28]:
#check for missing values
health.isna().sum()

,0
LSOA21CD,0
ah4dent,0
ah4gp,0
ah4hosp,0
ah4phar,0
ah4leis,0
ah4h,0


In [ ]:
#Read in Median Property Sales
m_sales = pd.read_csv('https://github.com/Jonathan-C-Barrett/GEOG5990M/raw/refs/heads/main/land-registry-house-prices-LSOA.csv')
m_sales.head()

In [21]:
#Select only columns relating to 2018
prop_16 = m_sales[['Code', 'Area','Year ending Mar 2016','Year ending Jun 2016', 'Year ending Sep 2016', 'Year ending Dec 2016']]
prop_16.head()

,Code,Area,Year ending Mar 2016,Year ending Jun 2016,Year ending Sep 2016,Year ending Dec 2016
0,NaN,NaN,NaN,NaN,NaN,NaN
1,E01000001,City of London 001A,910000,975000,999000,985000
2,E01000002,City of London 001B,880000,845000,842500,845000
3,E01000003,City of London 001C,630000,685000,745000,717500
4,E01000005,City of London 001E,570500,585000,556000,556000


In [29]:
#check for missing values
prop_16.isna().sum()

,0
Code,1
Area,1
Year ending Mar 2016,1
Year ending Jun 2016,1
Year ending Sep 2016,1
Year ending Dec 2016,1


In [31]:
#remove NA values
prop_16 = prop_16.dropna()
prop_16.isna().sum()

,0
Code,0
Area,0
Year ending Mar 2016,0
Year ending Jun 2016,0
Year ending Sep 2016,0
Year ending Dec 2016,0


In [ ]:
#examine property info
prop_18.info()

In [46]:
#combine health and property data by LSOA code in property to remove areas outside london
health_prop = pd.merge(health, prop_16, left_on='LSOA21CD', right_on='Code', how='right')
health_prop.head()

,LSOA21CD,ah4dent,ah4gp,ah4hosp,ah4phar,ah4leis,ah4h,Code,Area,Year ending Mar 2016,Year ending Jun 2016,Year ending Sep 2016,Year ending Dec 2016
0,E01000001,0.6328,1.0316,0.9581,0.7487,0.3596,-1.7834,E01000001,City of London 001A,910000,975000,999000,985000
1,E01000002,0.6230,1.3232,1.1370,0.6849,0.5571,-1.5523,E01000002,City of London 001B,880000,845000,842500,845000
2,E01000003,0.5709,0.8562,0.7862,0.5919,0.4608,-1.9813,E01000003,City of London 001C,630000,685000,745000,717500
3,E01000005,0.8337,1.1078,0.8697,0.6657,0.3717,-1.7260,E01000005,City of London 001E,570500,585000,556000,556000
4,E01032739,0.6641,1.6422,1.1970,0.6810,0.4148,-1.5235,E01032739,City of London 001F,599975,605500,582500,585000


In [47]:
#check for missing values
health_prop.isnull().sum()

,0
LSOA21CD,176
ah4dent,176
ah4gp,176
ah4hosp,176
ah4phar,176
ah4leis,176
ah4h,176
Code,0
Area,0
Year ending Mar 2016,0


In [39]:
#remove all rows outside of london
health_prop = health_prop['Code'].dropna()
health_prop.info()

<class 'pandas.core.series.Series'>
Index: 4659 entries, 0 to 31795
Series name: Code
Non-Null Count  Dtype 
--------------  ----- 
4659 non-null   object
dtypes: object(1)
memory usage: 72.8+ KB


In [45]:
# Data downloaded from https://geoportal.statistics.gov.uk/datasets/f23c50d2bffa4a8d9693bcf4ed72bdaf_0/explore?location=0.278866%2C-2.487028%2C0.00
shp =gpd.read_file('/content/Output_Areas_2021_EW_BFE_V9_-6709771492190738846.geojson')


In [ ]:
#inspect data
shp.head()

In [ ]:
#combine boundary data and health and property data through LSOA code
ldn_health_prop = pd.merge(shp, health_prop, left_on='LSOA21CD', right_on='lsoa_cd', how='left')
ldn_health_prop.head()

In [ ]:
#inspect data
ldn_health_prop.info()

In [49]:
#select only london boroughs
london_shp= ldn_health_prop.loc[ldn_health_prop['LSOA21NM'].str.contains('City of London') |
                    ldn_health_prop['LSOA21NM'].str.contains('Barking and Dagenham')|
                    ldn_health_prop['LSOA21NM'].str.contains('Barnet')|
                    ldn_health_prop['LSOA21NM'].str.contains('Bexley')|
                    ldn_health_prop['LSOA21NM'].str.contains('Brent') |
                    ldn_health_prop['LSOA21NM'].str.contains('Bromley')|
                    ldn_health_prop['LSOA21NM'].str.contains('Camden')|
                    ldn_health_prop['LSOA21NM'].str.contains('Croydon')|
                    ldn_health_prop['LSOA21NM'].str.contains('Ealing')|
                    ldn_health_prop['LSOA21NM'].str.contains('Enfield')|
                    ldn_health_prop['LSOA21NM'].str.contains('Greenwich')|
                    ldn_health_prop['LSOA21NM'].str.contains('Hackney')|
                    ldn_health_prop['LSOA21NM'].str.contains('Hammersmith and Fulham')|
                    ldn_health_prop['LSOA21NM'].str.contains('Haringey')|
                    ldn_health_prop['LSOA21NM'].str.contains('Harrow')|
                    ldn_health_prop['LSOA21NM'].str.contains('Havering')|
                    ldn_health_prop['LSOA21NM'].str.contains('Hillingdon')|
                    ldn_health_prop['LSOA21NM'].str.contains('Hounslow')|
                    ldn_health_prop['LSOA21NM'].str.contains('Islington')|
                    ldn_health_prop['LSOA21NM'].str.contains('Kensington and Chelsea')|
                    ldn_health_prop['LSOA21NM'].str.contains('Kingston upon Thames')|
                    ldn_health_prop['LSOA21NM'].str.contains('Lambeth')|
                    ldn_health_prop['LSOA21NM'].str.contains('Lewisham')|
                    ldn_health_prop['LSOA21NM'].str.contains('Merton')|
                    ldn_health_prop['LSOA21NM'].str.contains('Newham')|
                    ldn_health_prop['LSOA21NM'].str.contains('Redbridge')|
                    ldn_health_prop['LSOA21NM'].str.contains('Richmond upon Thames')|
                    ldn_health_prop['LSOA21NM'].str.contains('Southwark')|
                    ldn_health_prop['LSOA21NM'].str.contains('Sutton')|
                    ldn_health_prop['LSOA21NM'].str.contains('Southwark')|
                    ldn_health_prop['LSOA21NM'].str.contains('Tower Hamlets')|
                    ldn_health_prop['LSOA21NM'].str.contains('Waltham Forest')|
                    ldn_health_prop['LSOA21NM'].str.contains('Wandsworth')|
                    ldn_health_prop['LSOA21NM'].str.contains('Westminster'),:]

In [ ]:
#investigate data
london_shp.info()

In [ ]:
#explore on leaflet map
london_shp.explore()

In [ ]:
#check for missing values
london_shp.isna().sum()

This is an example block of mardown text I want to reference <a href="#ref1">[1]</a>. I might need to add some more citations <a href="#ref2">[2]</a><a href="#ref2">[3]</a>



Data Visualisation
- Don't forget to present your final two data visualisation (one spatial and one non-spatial) and the justifications about the decisions you made whilst preparing and visualising the data.

## References
<p><a href="add_url_here">[1]</a>Footnote citation goes here</p>
<p><a href="https://github.com/FrancescaPontin/GEOG5990">[2]</a>GEOG5990M course materials</p>
<p><a href="add_url_here">[3]</a>Citation 3</p>